Uno de los algoritmos basicos sobre el cual funciona la rama de machine learning es la *Regresion Lineal*. Como modo introductorio a Pytorch vamos a crear un modelo que sea capaz de predecir la produccion de manzanas y naranjas (variables objetivo) de diferentes regiones en base a la temperatura promedio, precipitaciones y la humedad (variables de entrada) en una region. Veamos la data de entranamiento.

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

En un modelo de regresion lineal cada variable objetivo es estimada como la suma de un peso por las variables de entrada mas una constante de sesgo conocida como bias.


```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

In [1]:
import numpy as np
import torch

# Data de entrenamiento

Si nos fijamos en la data de entrenamiento podriamos representarla como dos matrices: `entradas` y `objetivos`, con una fila por cada region y una columna por cada variable de la observacion. Para el caso de la tabla de arriba tendriamos.

In [2]:
# Entrada (temp, rainfall, humidity)
inputs = np.array([
    [73,67,43],
    [91,88,64],
    [87,134,58],
    [102,43,37],
    [69,96,70]], dtype='float32')

In [3]:
objetivos = np.array([
    [56,70],
    [81,101],
    [119,133],
    [22,37],
    [103,119]], dtype='float32')

Bien, hemos separado la data de entrenamiento en dos matrices debido a que vamos a necesitar operar con ellas de forma independiente. Se han creado como arreglo de Numpy debido a que tipicamente cualquier dataset que encontremos pueede ser leido en formato CSV directamente a arreglos de Numpy. **Entonces ahora debemos transformarlo a tensors.**

In [4]:
# Convertimos los arreglos de entrada a tensores
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(objetivos)
print(f'{inputs}')
print(f'{targets}')

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# Modelo de regresion lineal

Los pesos y los sesgos (`w11, w12, ... w23, b1 & b2`) pueden ser presentados como matrices inicializadas con valores aleatorios. La primera fila de `w` y el primer elemento de `b` son usados para predecir la primera variable objetivo, por ejemplo la produccion de manzanas.

In [5]:
# Se inicializan pesos y sesgos
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad = True)
print(w)
print(b)

tensor([[-0.6275,  0.2851, -0.3010],
        [-0.0155, -0.3856, -0.1424]], requires_grad=True)
tensor([ 0.1501, -0.9704], requires_grad=True)


`torch.randn` create un tensor de la formada dada con sus elementos escogidos aleatoriamente con una distribucion normal con media 0 y desviacion estandar 1.

El modelo para resolver el problema es simplemente una funcion que realiza una multiplicacion de matrices de entrada `inputs` y los pesos `w`(traspuestos) y luego le suma el sesgo `b`.

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

**Podemos definir el modelo en codigo de la siguiente forma**:

In [6]:
def model(x):
    return x @ w.t() + b

`@` representa una multiplicacion de matrices en Pytorch. El metodo `.t()` retorna la traspuesta de un tensor.

La matriz obtenida al pasar la data de entrada a nuestro moduelo genera una prediccion de las variables objetivos como se puede ver.

In [7]:
# Genera predicciones
preds = model(inputs)
print(preds)

tensor([[-39.4947, -34.0628],
        [-51.1223, -45.4304],
        [-33.6924, -62.2528],
        [-62.7274, -24.4036],
        [-36.8433, -49.0282]], grad_fn=<AddBackward0>)


Comparemos la prediccion de nuestro modelo con los objetivos reales de la data de entrenamiento.

In [8]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


Como se puede ver existe una gran diferencia entre las predicciones de nuestro modelos y los valores reales de las variables objetivo. Obviamente, esto se debe a que hemos inicializado los pesos y sesgos de nustro modelo con variables aleatorias. No podemos esperar que simplemente funcione asi.

# Funcion de perdida

Antes de mejorar nuestro modelo necesitamos un mecanismo para poder evaluar que tan bien o mal esta aproximando nuestro modelo comparado con los objetivos reales, para ello vamos a calcular el error cuadratico medio **MSE**.

In [9]:
def mse(t1,t2):
    diff = t1 - t2
    return torch.sum(diff*diff) / diff.numel()

El metodo `torch.sum` retorna la suma de todos los elementos en un tensor, y el metodo `.numel` retorna la cantidad de elementos en un tensor. Vamos a calcular el error cuadrado medio para la prediccion actual de nuestro modelo.

In [10]:
loss = mse(preds,targets)
print(loss)

tensor(17902.3887, grad_fn=<DivBackward0>)


Para este caso el error dio 15344. Esto se puede interpretar como que cada elemento difiere del objetivo real por 123 unidades (raiz cuadrada de 15344). Claramente el error esta demasiado alto, por lo cual debemos buscar una forma de reducirlo.

# Calcular gradientes

Con Pytorch podemos automaticamente calcular el gradiente o la derivada de la funcion de perdida con respecto a los pesos y el sesgo debido a que `requires_grad` esta seteado como `True`.

In [11]:
# Calcular gradientes
loss.backward()

Los gradientes se almacenan en el atributo `.grad` del tensor respectivo. En este caso la derivada con respecto a los pesos es en si una matriz con las mismas dimensiones.

In [12]:
print(w)
print(w.grad)

tensor([[-0.6275,  0.2851, -0.3010],
        [-0.0155, -0.3856, -0.1424]], requires_grad=True)
tensor([[-10113.9746, -11110.7871,  -6868.4419],
        [-11153.1748, -12958.6055,  -7840.9648]])


La funcion de perdida es una funcion cuadratica de los pesos y sesgos. Nuestro objetivo es encontrar un conjunto de pesos y bias para los cuales la funcion alcance su minimo posible. Si graficamos la funcion de perdida con respecto a cualquier peso individual o sesgo nos encontraremos como una figura como la siguiente.

![postive-gradient](https://i.imgur.com/hFYoVgU.png)

En donde, si el **gradiente del elemento es positivo** entonces disminuir ligeramente el valor del elemento disminuira la perdida. 

Por otro lado podemos encontrar casos como:

![negative=gradient](https://i.imgur.com/w3Wii7C.png)

En donde si el **gradiente del elemento es negativo** entonces aumentar ligeramente el valor del elemento disminuira la perdida.


Aumentar o disminuir la funcion de perdida al cambiar los pesos de un elemento proporcional a su gradiente es la base del algoritmo de optimizacion que vamos a usar para mejorar nuestro modelo.

Antes de proceder vamos a resetear los gradientes de los tensors a cero. Para ello vamos a llamar al metodo `.zero()`.**Necesitamos hacer esto debido a que Pytorch acumula los gradientes**. Por ejemplo la proxima vez que llamemos al metodo `.backward` de la funcion de perdida, los nuevos valores del gradiente seran sumados con los valores existentes lo cual podria llevar a resultados inesperados.

In [13]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


# Ajustar los pesos y el sesgo utilizando descenso de gradiente

Ahora vamos a reducir el riesgo y mejorar nuestro modelo utilizando la optimizacion por decsenso de gradiente siguiendo los siguientes pasos:

1. Generar la prediccion.
2. Calcular la perdida.
3. Calcular los gradientes con respecto a los pesos y el sesgo.
4. Ajustar los pesos al restarles una pequeña proporcion del gradiente.
5. Resetear los gradientes a cero.

In [14]:
# Generar las predicciones
preds = model(inputs)
print(preds)

tensor([[-39.4947, -34.0628],
        [-51.1223, -45.4304],
        [-33.6924, -62.2528],
        [-62.7274, -24.4036],
        [-36.8433, -49.0282]], grad_fn=<AddBackward0>)


In [15]:
# Calcular la perdida
loss = mse(preds, targets)
print(loss)

tensor(17902.3887, grad_fn=<DivBackward0>)


In [16]:
# Calcular gradientes
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-10113.9746, -11110.7871,  -6868.4419],
        [-11153.1748, -12958.6055,  -7840.9648]])
tensor([-120.9760, -135.0356])


In [17]:
# Ajustar pesos y sesgos
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

- Se usa `torch.no_grad` para indicarle a Pytorch que no deberia hacer un seguimiento, calculo o modificacion de los gradientes luego de actualizar los pesos y sesgos.
- Se multiplican los gradientes por un numero muy pequeño, para asegurarnos de no hacer un cambio muy brusco ya que queremos aproximarnos poco a poco a la solucion. **Esto se conoce como tasa de aprendizaje**.
- Despues de actualizar los pesos resetiamos los gradientes a cero para evitar afectar calculos futuros.

Veamos como se ven los pesos y sesgos.

In [18]:
print(w)
print(b)

tensor([[-0.5263,  0.3962, -0.2323],
        [ 0.0960, -0.2560, -0.0640]], requires_grad=True)
tensor([ 0.1513, -0.9690], requires_grad=True)


Con los nuevos pesos y sesgos, el modelo disminuye el error.

In [19]:
preds = model(inputs)
loss = mse(preds,targets)
print(loss)

tensor(12195.7051, grad_fn=<DivBackward0>)


Claramente hemos logrado disminuir significativamente la funcion de perdida. Ahora solo bastaria realizar este proceso iterativamente.

# Entrando el modelo

Para reducir aun mas la funcion de perdida, lo que debemos hacer ahora es iterar multiples veces. **Cada iteracion es lo que conocemos como una epoca**. En este caso vamos a entrenar el modelo por 1000 epocas.

In [20]:
for i in range(1000):
    preds = model(inputs)
    loss = mse(preds,targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Ahora verificamos que la funcion de perdida sea menor.

In [21]:
preds = model(inputs)
loss = mse(preds,targets)
print(loss)

tensor(6.1004, grad_fn=<DivBackward0>)


Como se puede ver, la funcion de perdida ha disminuido considerablemente. Ahora vamos a ver las predicciones del modelo y compararlas con los objetivos de la data de entrenamiento.

In [22]:
preds

tensor([[ 57.4012,  70.3532],
        [ 80.4624,  99.1743],
        [122.2816, 136.2866],
        [ 22.1098,  38.0585],
        [ 98.3163, 115.7360]], grad_fn=<AddBackward0>)

In [23]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

Como podemos ver, la prediccion es muy cercana a las variables objetivo. Se podrian obtener aun mejores resultados entrenando el modelo por mas epocas.